# Project 3 Report
***Chandler Haukap and Hassan Saad***

In this project, we simulated several different events that could occur in Minesweeper. The `main.py` file contains the code that:
* Creates the playing board
* Simulates the location of mines specific to a game session
* Allows for events such as clicking on a cell (check), flagging a cell (flag), or seeing the solution of the board (solution)
    * The flag and check events allow for metadata to be created depending on if the specific cell contains an underlying mine or is a "safe spot"
    
The game interface consists of four endpoints:
* /home is where the user requests a new game. This endpoint returns a unique UUID session ID that the user must submit to the other endpoints.
    * parameters: none
    * returns: A session identifier
* /flag is how the user flags a location as having a bomb. If the space is not a bomb, the user loses (at least, in our version of Minesweeper)
    * parameters: x, y, session_id
    * returns: "Correct" if the space is a bomb, or "Incorrect! Game over." if the space is not a bomb
* /check is how the user tests a space. If the space doesn't have a bomb, they are safe. If it does have a bomb, they lose.
    * parameters: x, y, session_id
    * returns: "Correct" if the space is not a bomb, or "Incorrect! Game over." if the space is a bomb
* /solution If the user wants to give up they can get the solution via this endpoint. It returns an n x n grid representing the game board.
    * parameter: session_id
    * returns: The game board

## Explanation of Data Pipeline

* Before we were able to do anything, we had to modify an existing `docker-compose.yml` file and updated the "mids" image so that we can have the latest version of Redis running in our Minesweeper API. You can view the image [on DockerHub](https://hub.docker.com/layers/180930452/hassansaadca/saad_project3/latest/images/sha256-e44f736674fb069e48aa8c2d1ecf072e5e5be26cfa2d1948b0284f59ddc5c6c2?context=repo).
    
    Now we're ready to run the API and store/access the data.
    
* First, the data gets created by the the Flask API, i.e. our `main.py` file, with the help of a python file that simulates gameplay (`event_generator.py`). With each event described above, data is generated in string format and fed into a Kafka queue. 
* Next, there are 3 ways in which we can log this data to a parquet file within the Hadoop environment. We explain these processes below, but for a summary:
    1. We generate the data, then open a Jupyter notebook within the pyspark environment, and we use Pyspark to create the parquet file.
    2. We generate the data, then run a Python file within the pyspark environment which takes a batch of data from the Kafka queue and writes it to a new parquet file.
    3. We run a python file that runs an infinite loop and has the ability to continuously stream data to the Hadoop environment ever few seconds. As we run the `generate_events.py` file, this data gets automatically fed to the same parquet file.
* Finally, we query the data within the Pyspark notebook (below)

Below is a diagram of how our pipeline works:

<img src="205_p3_pipeline.png">

## Part 1: Setup of the Data Pipeline and Getting the Flask API Running

We do all of this within the working directory/ repository within our GCP VM.

`~w205/mids-205-project-3/`

**Spin up Docker Container:**

`docker-compose up -d`

**Start the flask API** 

`docker-compose exec mids env FLASK_APP=/w205/mids-205-project-3/main.py flask run --host 0.0.0.0`

In a new shell terminal, **Create a Kafka Topic called 'events':**

`docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --bootstrap-server kafka:29092`


In a new shell terminal, **set up to watch the incoming Kafka Queue:** Remember to navigate to the working directory first.

`docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning`

-----

# Part 2: Different ways of accessing and storing data


### Method 1: Using the Jupyter Notebook in the Pyspark Environment

In this section, we'll use the most basic event in our API, the `solution` event. This is a situation in which a player checks to see where the mines are on the board. We'll filter for this event and write all the data points to a parquet file within the Jupyter notebook environment.

After we've set up Kafka to watch the incoming queue, we can **run the command to generate data**

`docker-compose exec mids python /w205/mids-205-project-3/generate_events.py`

This will simulate 5 game sessions, and within each game the user "peeks" at the solution 5 times (so we get a total of 25 data points).

Next, we have to access the Jupyter notebook after making the data from Kafka available within the same environment.

We create a symbolic link to our working directory inside the Pyspark environment:

`docker-compose exec spark bash`

`ln -s /w205/mids-205-project-3 project3data`

`exit`

Then we launch a jupyter notebook, again inside the Pyspark container, using our VM external IP address:

`docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark`

In [10]:
import json
from pyspark.sql import Row
from pyspark.sql.functions import udf

In [11]:
spark

In [12]:
raw_events = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load() 

In [13]:
raw_events.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 65 76 65 6...|events|        0|     0|2021-12-08 21:57:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     1|2021-12-08 21:57:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     2|2021-12-08 21:57:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     3|2021-12-08 21:57:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     4|2021-12-08 21:57:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     5|2021-12-08 21:57:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     6|2021-12-08 21:57:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     7|2021-12-08 21:57:...|            0|
|null|[7B 22 65 76 65 6...|events|        0

At this stage we can check to make sure that our data is of the correct size. For each session we generate 1 visit to the home page, and 100 to the solution, check, and flag pages totalling 301 events per session.

In [14]:
raw_events.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [25]:
print(f'Number of total events created: {raw_events.count()}')

Number of total events created: 1505


In [16]:
all_events = raw_events.select(raw_events.value.cast('string'))

An example of a data point that was fed to the Kafka queue via the API

In [17]:
json.loads(all_events.collect()[-1].value)

{'event_type': 'solution',
 'session_id': 'b9e269a3-644a-4c3e-84a8-8e2b641f0537'}

Define a function that we will use to filter for events of a specific type:

In [18]:
name = 'solution'

@udf('boolean')
def test(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == name:
        return True
    return False

Filter the data points from the raw events, extracting only the solution_events.

In [19]:
solution_events = raw_events \
    .select(raw_events.value.cast('string').alias('stats'),\
            raw_events.timestamp.cast('string'))\
    .filter(test('stats'))

In [20]:
extracted_solution_events = solution_events \
    .rdd \
    .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.stats))) \
    .toDF()

In [21]:
extracted_solution_events.show(5)

+----------+--------------------+--------------------+
|event_type|          session_id|           timestamp|
+----------+--------------------+--------------------+
|  solution|e4400ca4-67d9-406...|2021-12-08 21:57:...|
|  solution|e4400ca4-67d9-406...|2021-12-08 21:57:...|
|  solution|e4400ca4-67d9-406...|2021-12-08 21:57:...|
|  solution|e4400ca4-67d9-406...|2021-12-08 21:57:...|
|  solution|e4400ca4-67d9-406...|2021-12-08 21:57:...|
+----------+--------------------+--------------------+
only showing top 5 rows



Write this table to a Parquet file:

In [22]:
extracted_solution_events \
    .write \
    .mode('overwrite') \
    .parquet('/tmp/solution_requests')

Load the data from the same Parquet file:

In [23]:
solution_batch = spark.read.parquet('/tmp/solution_requests')

Create a table that we can use to run PySpark SQL commands:

In [24]:
solution_batch.registerTempTable('solution_requests_table')

Query the Table:

In [23]:
solutions = spark.sql("select * from solution_requests_table").toPandas()
solutions.head()

,event_type,session_id,timestamp
0,solution,be0e3368-fdef-4861-9800-81c2e2183598,2021-12-07 23:42:20.509
1,solution,be0e3368-fdef-4861-9800-81c2e2183598,2021-12-07 23:42:20.524
2,solution,be0e3368-fdef-4861-9800-81c2e2183598,2021-12-07 23:42:20.538
3,solution,be0e3368-fdef-4861-9800-81c2e2183598,2021-12-07 23:42:20.554
4,solution,be0e3368-fdef-4861-9800-81c2e2183598,2021-12-07 23:42:20.568


How many different sessions were started? We can run a pandas command to just check for unique values of session_id:

In [27]:
print(f'Number of Unique Sessions: {solutions.session_id.unique().shape[0]}\n')
print(f'Unique Sessions:')
for i in solutions.session_id.unique():
    print(i)

Number of Unique Sessions: 5

Unique Sessions:
be0e3368-fdef-4861-9800-81c2e2183598
36412095-d22e-4908-b16f-219f6e48a473
31034adb-8130-44f7-bc1c-d8cdcd41b567
0700aed7-a236-4855-b729-24a21964ee14
191d876c-1f14-4deb-bfb8-b469433059fd


###  Method 2: Using a python file within the pyspark environment

In this section, we'll use the other events in our API, the `flag` and `check` events. This `flag` event is a situation in which a player thinks there is a mine under a cell, so they flag it (usually with a right-click) as a warning and avoid uncovering it. The `check` event is a situation in which a player uncovers (left-clicks) a cell. If a mine is underneath the cell, the player loses, and if the cell is not a mine, the number of mines surrounding that cell is displayed.

We can use the same data that we had generated before. Recall this was done by running this command above while using method 1:

`docker-compose exec mids python /w205/mids-205-project-3/generate_events.py`

Again, this simulated 5 different game sessions, and in each game, the user marked 60 potentially dangerous cells and uncovered 50 cells. We expect to see 500 `flag` events and 500 `check` events in our data set.


After having generated the data, we run the `write_flag_events_batch.py` and `write_check_events_batch.py` files while inside the Pyspark environment. If we've already used Method 1, we don't need to create a symbolic link to our working directory again.

`docker-compose exec spark bash`

`cd project3data`

`python write_flag_events_batch.py`

This will create a parquet file for each event within our Hadoop environment: `/tmp/flag_cell` and `/tmp/check_cell`

We can exit from the spark container shell and check the Hadoop environment file system with the following command:

`docker-compose exec cloudera hadoop fs -ls /tmp/` 

...which yields the previous `solution_request` parquet file, the new `flag_cell` file and `check_cell` files as well.

Now we can load these parquet files and query them with PySpark and Pandas.

In [6]:
flag_batch = spark.read.parquet('/tmp/flag_cell')

In [7]:
flag_batch.registerTempTable('flag_cell_table')

In [8]:
flags = spark.sql("select * from flag_cell_table").toPandas()
flags.head()

,event_type,outcome,session_id,timestamp,x_coord,y_coord
0,flag,incorrect,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:33.568,0,53
1,flag,incorrect,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:33.577,36,98
2,flag,incorrect,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:33.588,99,47
3,flag,incorrect,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:33.597,73,37
4,flag,incorrect,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:33.607,46,40


Let's confirm that we did indeed write all 500 flag events in the Kafka Queue:

In [9]:
print(f'Number of flag events logged in Parquet file: {flags.shape[0]}')

Number of flag events logged in Parquet file: 500


Now let's see what percentage of flags are put in appropriate spots and how many were placed in incorrect locations:

In [41]:
incorrect_percentage = 100* flags[flags.outcome == 'incorrect'].shape[0]/flags.shape[0]
correct_percentage = 100* flags[flags.outcome == 'correct'].shape[0]/flags.shape[0]

In [48]:
print(f'Percentage of Correct Flags: {correct_percentage}5')
print(f'Percentage of Incorrect Flags: {incorrect_percentage}%')

Percentage of Correct Flags: 17.05
Percentage of Incorrect Flags: 83.0%


Let's see the same distribution for each session:

In [49]:
100*flags.groupby('session_id').outcome.value_counts()/flags.groupby('session_id').outcome.count()

session_id                            outcome  
0700aed7-a236-4855-b729-24a21964ee14  incorrect    83.333333
                                      correct      16.666667
191d876c-1f14-4deb-bfb8-b469433059fd  incorrect    86.666667
                                      correct      13.333333
31034adb-8130-44f7-bc1c-d8cdcd41b567  incorrect    81.666667
                                      correct      18.333333
36412095-d22e-4908-b16f-219f6e48a473  incorrect    80.000000
                                      correct      20.000000
be0e3368-fdef-4861-9800-81c2e2183598  incorrect    83.333333
                                      correct      16.666667
Name: outcome, dtype: float64

Now let's run a similar process with the check functions. We'll query to see what percentage of uncovered cells had underlying mindes and what cells were safe (aggregated by session ID). 

In [1]:
check_batch = spark.read.parquet('/tmp/check_cell')

In [2]:
check_batch.registerTempTable('check_cell_table')

In [3]:
checks = spark.sql("select * from check_cell_table").toPandas()
checks.head()

,event_type,neighboring_bombs,outcome,session_id,timestamp,x_coord,y_coord
0,check,26,safe,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:31.983,66,82
1,check,62,safe,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:31.999,1,66
2,check,12,safe,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:32.022,84,95
3,check,11,hit_mine,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:32.041,82,14
4,check,43,safe,e4400ca4-67d9-406d-abb6-a4e68c81e472,2021-12-08 21:57:32.061,42,40


Let's check to see that we have 500 check events (100 per each of the 5 sessions.)

In [5]:
print(f'Number of check events logged in Parquet file: {checks.shape[0]}')

Number of check events logged in Parquet file: 500


In [4]:
100*checks.groupby('session_id').outcome.value_counts()/checks.groupby('session_id').outcome.count()

session_id                            outcome 
17ba600a-0ac8-42df-94af-14fd53d84222  safe        85.0
                                      hit_mine    15.0
1d9c8840-6e20-49a9-8928-0f9b13de5be2  safe        81.0
                                      hit_mine    19.0
274f89df-d1c5-4c20-b338-1be0813a5ee3  safe        82.0
                                      hit_mine    18.0
b9e269a3-644a-4c3e-84a8-8e2b641f0537  safe        79.0
                                      hit_mine    21.0
e4400ca4-67d9-406d-abb6-a4e68c81e472  safe        79.0
                                      hit_mine    21.0
Name: outcome, dtype: float64

###  Method 3: Writing Data to parquet files in a streaming manner

In this section, we'll use the last event in our API( the `check` event) again, just to demonstrate how live streaming of events work. 

This time, we run a python file within the spark environment in preparation to read incoming data from the Kafka queue and write it to a parquet file.

We enter the spark container shell... 

`docker-compose exec spark bash`

...and run the following commands:

`cd project3data`

`python write_check_events_stream.py`

Note that we run this file before we start producing the data. This runs until interrupted manually. 

*This is in comparison method 2, for which we generated data, then we wrote it in batches to parquet files.*

After this, we test to see if it works by now calling the `generate_events.py` file from within our original working directory (i.e. `~/w205/mids-205-project-3`). 

Once we run this data generating code, `write_check_events_stream.py` adds to a parquet file to the Hadoop environment as data comes into the Kafka queue, which we could then load as a PySpark SQL table by running the commands below.


In [61]:
check_batch = spark.read.parquet('/tmp/check_stream_data')

In [2]:
check_batch.registerTempTable('flag_cell_table')

From this point, in the same manner as we've done above, the flag_cell_table is available for querying through PySpark SQL.